# **Advanced DDL & Window Queries**

In this notebook we will continue to build on the DDL concepts that we learned last week. We will also learn to write window queries to simplify data retrieval.

## **Advanced DDL**

### **CREATE Statements with Constraints**

Last week we learned to write CREATE statements to add a new table to a database. When writing a CREATE script we can do more than specify the columns and data types. We can also establish a primary key, add a foreign key, and implement other constraints. This makes our database tables more organized and secure. It is also crucial for maintaining data integrity. <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">In addition to adding constraints, we can also implement useful features like an auto-incrementing value for a primary key and variables to hold values for later use.</span>

<span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;"><br></span>

<span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Throughout this notebook we will write scripts to create and load database tables to manage information for a hotel. We will start by creating a guest table where we define an auto-incrementing primary key within our CREATE statement. We will also use the keywords <i>UNIQUE </i>and <i>NOT NULL</i>&nbsp;to ensure that some columns do not allow duplicate or null values. Below is a breakdown of what the new keywords in this query do:</span>

- <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">IDENTITY- A function that generates values for a column automatically based on two arguments, the seed and increment. The seed is the starting value and the increment is the amount it increases by. So, if we used IDENTITY(3,4) the first value would be 3, the next would be 7, and then 11. This is how we make our primary keys increase automatically without relying on subqueries.</span>
    - <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">If a record is deleted or rolled back, the system will not reuse the number that was generated for the missing record. There will just be a gap in the numbering.</span>
- <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">PRIMARY KEY: This keyword marks an attribute as being the primary key for a table. Marking an attribute as primary key does the following:</span>
    - <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Ensure uniqueness and does allow duplicate values</span>
    - <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Does not allow null values to be inserted into the field</span>
    - <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Creates an index on the field (we will learn more about this soon)</span>
- UNIQUE: When a field is marked as UNIQUE, scripts that would insert a duplicate value into the field will result in an error. This ensures data integrity where values must be unique.
- NOT NULL: Marking a field as NOT NULL will make it so that scripts that would insert a null value into the field will result in an error.

We use these commands by specifying them for relevant attributes _after_ we specify the data type.

In [ ]:
-- Create Guests table
CREATE TABLE ho_guest (
    guest_id INT IDENTITY(1,1) PRIMARY KEY, --Assign guest_id as the primary key for the table
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    phone_number VARCHAR(15) UNIQUE, --Ensure that all phone_numbers are unique
    email VARCHAR(100) UNIQUE NOT NULL, --Ensure that all emails are unique and that there are no null values
    address VARCHAR(255)
);

Now we can insert some data into the guest table. Note that we do not have to include a value for guest\_id since it is setup as an identity.

In [ ]:
BEGIN TRANSACTION;
    BEGIN TRY
        INSERT INTO ho_guest VALUES
        ('John', 'Doe', '123-456-7890', 'john.doe@example.com', '123 Maple Street'),
        ('Jane', 'Smith', '987-654-3210', 'jane.smith@example.com', '456 Oak Avenue'),
        ('Robert', 'Brown', '555-123-4567', 'robert.brown@example.com', '789 Pine Road'),
        ('Emily', 'Davis', '444-555-6666', 'emily.davis@example.com', '101 Birch Lane'),
        ('Michael', 'Wilson', '333-222-1111', 'michael.wilson@example.com', '202 Cedar Street'),
        ('Jessica', 'Taylor', '777-888-9999', 'jessica.taylor@example.com', '303 Spruce Drive'),
        ('David', 'Anderson', '888-777-6666', 'david.anderson@example.com', '404 Elm Court');
        COMMIT;
    END TRY
    BEGIN CATCH
        PRINT 'Error message: ' + ERROR_MESSAGE();
        ROLLBACK;
    END CATCH;

In [ ]:
--View inserted records
SELECT * FROM ho_guest

In the below cell we will demonstrate what happens when an insertion for a table with an identity column fails.

In [ ]:
BEGIN TRANSACTION;
    BEGIN TRY
        INSERT INTO ho_guest VALUES
        ('Kayla', 'Lemon', '573-673-7583', 'kayla.lemon.com', '872 Oyster Street'),
        ('Jason', 'File', '123-456-7890', 'jason.file@example.com', '345 Maple Street'); --insert will fail because phone number is not unique
        COMMIT;
    END TRY
    BEGIN CATCH
        PRINT 'Error message: ' + ERROR_MESSAGE();
        ROLLBACK;
    END CATCH;

The inserted records would have had a guest\_id of 8 and 9 because those were next in the identity insert. Since the transaction failed, the id numbers that were generated for each record will no longer be used. They were "used up"  The below cell corrects the issue, let's insert the data and observe the results.

In [ ]:
BEGIN TRANSACTION;
    BEGIN TRY
        INSERT INTO ho_guest VALUES
        ('Kayla', 'Lemon', '573-673-7583', 'kayla.lemon.com', '872 Oyster Street'),
        ('Jason', 'File', '986-456-7890', 'jason.file@example.com', '345 Maple Street'); --updated phone number to be unique
        COMMIT;
    END TRY
    BEGIN CATCH
        PRINT 'Error message: ' + ERROR_MESSAGE();
        ROLLBACK;
    END CATCH;

In [ ]:
--View inserted data
SELECT * FROM ho_guest

We can see from the above results that id numbers 8 and 9 were skipped. It is important that you have an understanding of the kind of behaviors that can result in gaps in identity id numbers, and that an id number isn't always the best indicator of how many records are in a table.

### ✏️ **Practice**

Script a CREATE statement to add an ho\_room table to your database with the following specifications:

- room\_id (Primary Key) Automatically increment by 1 starting at 1
- room\_number (Unique, Not Null)
- room\_type (e.g., Single, Double, Suite)
- price\_per\_night
- availability (BIT)

Afterward, script a transaction that inserts the following data into the table:
|room_id|room_number|room_type|price_per_night|availability|
|---|---|---|---|---|
|1|101|Single|75.00|1|
|2|102|Single|75.00|0|
|3|103|Double|120.00|1|
|4|104|Double|120.00|0|
|5|201|Suite|200.00|1|
|6|202|Suite|200.00|0|
|7|203|Single|75.00|1|
|8|204|Double|120.00|1|
|9|301|Suite|250.00|1|
|10|302|Single|80.00|0|


In [ ]:
--Create room table
--Your code here

In [ ]:
--Insert data to room table
--Your code here

Now that we have a couple of tables to work with, we can setup a table that has foreign key constraints. Foreign key constraints ensure that a reocrd cannot be inserted into a table if there is not a related record in another table. For this example we will be adding a reservation table. Each reservation must be related to a guest and a room, so we will be adding foreign key constraints to the room\_id and guest\_id attributes. We do not define foreign keys the same way as primary keys. To establish an attribute as a foreign key we add a REFERENCE at the end of the CREATE statement that indicates what table and column the foreign key is related to. We do this by using the keyword CONSTRAINT, giving the constraint a name (usually starts with fk and includes both the source and referenced table/column) and then the syntax for defining the foreign key.

In [ ]:
-- Create Reservations table
CREATE TABLE ho_reservation (
    reservation_id INT IDENTITY(1,1) PRIMARY KEY,
    guest_id INT,
    room_id INT,
    check_in_date DATE,
    check_out_date DATE,
    total_amount DECIMAL(10, 2),
    CONSTRAINT fk_reservation_guest_id_ref_guest_guest_id FOREIGN KEY (guest_id) REFERENCES ho_guest(guest_id), --guest_id is a foreign key that references the guest_id in ho_guest
    CONSTRAINT fk_reservation_room_id_ref_room_room_id FOREIGN KEY (room_id) REFERENCES ho_room(room_id) ----room_id is a foreign key that references the room_id in ho_room
);

The above script creates a reservation table that ensures that each guest and room id entered into this table exists in their referenced tables. If we attempt to add data into the tables that violates the foreign key constraint, then we will get an error:

In [ ]:
--Insert data into ho_reservation
BEGIN TRANSACTION;
    BEGIN TRY
        INSERT INTO ho_reservation VALUES
        (1, 1, '2024-07-01', '2024-07-05', 300.00),
        (2, 13, '2024-07-02', '2024-07-06', 300.00); --The room_id we are trying to insert is not present in the ho_room table
        COMMIT;
    END TRY
    BEGIN CATCH
        PRINT 'Error message: ' + ERROR_MESSAGE();
        ROLLBACK;
    END CATCH;

The above cell results in an error because we attempted to insert a value into the table that violates the foreign key constraint that we established. To fix this we can either change the room\_id to one that is in the room table, or we can add the room\_id (13) to the room table. For simplicity sake we will just change the room\_id we are inserting.

In [ ]:
--Insert data into ho_reservation
BEGIN TRANSACTION;
    BEGIN TRY
        INSERT INTO ho_reservation VALUES
        (1, 1, '2024-07-01', '2024-07-05', 300.00),
        (2, 2, '2024-07-02', '2024-07-06', 300.00); --The room_id we are trying to insert is not present in the ho_room table
        COMMIT;
    END TRY
    BEGIN CATCH
        PRINT 'Error message: ' + ERROR_MESSAGE();
        ROLLBACK;
    END CATCH;

In [ ]:
--View records to verify insertion
SELECT * FROM ho_reservation

Beyond foreign key constraints, there are other types of constraints that we can implement. Below is a CREATE statement to add a services table to the hotel database. This statement includes a variety of constraints for demonstration.

In [ ]:
-- Create Services table
CREATE TABLE ho_services (
    service_id INT PRIMARY KEY, -- Primary key
    service_name VARCHAR(100) NOT NULL UNIQUE, -- Ensure service names are unique and not null
    service_description VARCHAR(255),
    price DECIMAL(10, 2) NOT NULL CHECK (price >= 0.00), -- Ensure price is non-negative
    max_capacity INT DEFAULT 10 CHECK (max_capacity > 0 AND max_capacity <= 100), -- Default value and range constraint
    available_from TIME,
    available_to TIME,
    CHECK (available_from < available_to) -- Ensure the available_from time is before the available_to time
);

In the above cell we use several different kinds of constraints. Below is a short summary of each:

- DEFAULT- Defines a default value for the attribute
- CHECK- This keyword should be paired with a condition. If the condition evaluates to true, the script will execute normally, if the condition evaluates to false the query will return an error. If multiple CHECKs are present then they are evaluated in the order they are listed. You can use logical operators like AND/OR/NOT with your conditions if you need to assess multiple conditions in one CHECK.

## **More About DROP Statements**

We can add a special syntax to our DROP statements to make them more dynamic. By using IF EXISTS we will only try to run the DROP operation if the table actually exists. In the code cell below we try to drop a non-existent table the way we did last week, which results in an error:

In [ ]:
--Drop table incorrectly
DROP TABLE ho_parking

The below cell implements IF EXISTS, which prevents the error from occuring:

In [ ]:
--Drop table correctly
DROP TABLE IF EXISTS ho_parking


## **Working with Indexes**

Indexes are a data structure that improves the speed of data retrieval, but at the cost of additional _write_ operations and storage space to maintain the index. When we create an index, it is a seperate data structure from our table. It reorganizes the data in a way that makes it faster to sort through.

**Clustered Index:** Each table can only have one clustered index, and this defines the order that data is stored in the table or object. This determines the default order that data is sorted through when queried. This is usually the primary key.

**Un-Clustered Index:** You can have as many of these as you want for each of your tables. These indexes divide data up into categories so that the database engine can identify all data in a category, and then sort through that for what it needs instead of all records.

The best way to explain is with an example. Instead of a database, imagine that we have paper files in a filing cabinet. We have a file for each of our customers that are sorted in our filing cabinet by customer ID number. This is our main filing cabinet and could be thought of as using the clustered index. Everytime we want to find a customer, we start at the beginning of this filing cabinet and flip through each file until we find the one we are looking for.

There are often times that we need to pull out all customer files for specific regions. To make things easier, we bring in a second filing cabinet that has records grouped by region. The second filing cabinet is an un-clustered index on the region field.

It is your job to pull out the customer file for each customer in Region A. Would it be faster to sort through each record one by one in the first filing cabinet, or search for the section containing all records from Region A in the second filing cabinet (since they are already sorted)? The fastest way would be to use the second cabinet.

Using the second cabinet made it much easier to pull out the needed data, but it takes time to keep it sorted and it takes up more space in the storage closet because it is a completely seperate cabinet. This is how databases work. Each index you add is like adding a whole new version of that table that is sorted differently. It can make it much faster to pull out data, but it takes up more space and processing power because each time there is a change, the "cabinet" needs to resort all of its files so they are in order again.

Now, after that lengthy explanation, here is the code that you can use to setup a field as a non-clustered index. We will be using the phone number field for guests:

In [ ]:
CREATE NONCLUSTERED INDEX IDX_ho_guest_phone_number --We name the index in the style IDX_TableName_ColumnName
ON ho_guest (phone_number) --We use the ON operator to specify which table and column(S) we want to create the index for

Normally we could write a couple of queries that would demonstrate the time difference in searching for a specific phone number before and after implementing the index, but the tables we are using don't have enough data for our index to make a noticable difference. Any percieved difference will be due to connection speed, not indexing.

### **✏️ Practice**

In the below code cell, implement an unclustered index on room number in the room table.

In [ ]:
--You code here
CREATE

## **Set Operations**

There are often times when we will need to combine or compare the results of two select statements. We could do this using JOINs or subquerying, but SQL provides _set operators_ to make these tasks a little more straightforward. Below are the set operators we will be exploring:

- UNION: Merges all unique rows from two or more SELECT statements, eliminating duplicates.
- UNION ALL: Merges all rows from two or more SELECT statements, keeping duplicates.
- INTERSECT: Returns only the rows that appear in both SELECT statements.
- EXCEPT: Returns rows from the first SELECT statement that don't appear in the second.

The simple hotel system is not the best database to demonstrate these operations, so we will be shifting to the storage unit databse.

  

Below are two cells with SELECT statements that each select different sets of data, with the same schema (columns):

In [ ]:
--SELECT statement #1
SELECT c.first_name, c.last_name, c.phone_number, u.unit_number, f.city
FROM su_customer as c
JOIN su_rental as r ON c.customer_id = r.customer_id
JOIN su_unit as u ON r.unit_id = u.unit_id
JOIN su_facility as f ON u.facility_id = f.facility_id
JOIN su_invoice as i ON r.rental_id = i.rental_id
WHERE i.amount > 400

In [ ]:
--SELECT statement #2
SELECT c.first_name, c.last_name, c.phone_number, u.unit_number, f.city
FROM su_customer as c
JOIN su_rental as r ON c.customer_id = r.customer_id
JOIN su_unit as u ON r.unit_id = u.unit_id
JOIN su_facility as f ON u.facility_id = f.facility_id
WHERE r.discount_amount > 0

The first query returns information about rentals where the invoice amount is greater than $400, while the second query returns the same information for rentals where there was a discount amount greater than $0. We are going to use each of our set operators with these two queries and evaluate the results.

  

Starting with a UNION, this will put all of the records from both statements together _with no duplicates:_

In [ ]:
--SELECT statement #1
SELECT c.first_name, c.last_name, c.phone_number, u.unit_number, f.city
FROM su_customer as c
JOIN su_rental as r ON c.customer_id = r.customer_id
JOIN su_unit as u ON r.unit_id = u.unit_id
JOIN su_facility as f ON u.facility_id = f.facility_id
JOIN su_invoice as i ON r.rental_id = i.rental_id
WHERE i.amount > 400

UNION

--SELECT statement #2
SELECT c.first_name, c.last_name, c.phone_number, u.unit_number, f.city
FROM su_customer as c
JOIN su_rental as r ON c.customer_id = r.customer_id
JOIN su_unit as u ON r.unit_id = u.unit_id
JOIN su_facility as f ON u.facility_id = f.facility_id
WHERE r.discount_amount > 0

The above return has some duplicate customers, but only because they have different storage units. Each individual row is indeed unique. If we use UNION ALL, then we will see true duplicates in the return:

In [ ]:
--SELECT statement #1
SELECT c.first_name, c.last_name, c.phone_number, u.unit_number, f.city
FROM su_customer as c
JOIN su_rental as r ON c.customer_id = r.customer_id
JOIN su_unit as u ON r.unit_id = u.unit_id
JOIN su_facility as f ON u.facility_id = f.facility_id
JOIN su_invoice as i ON r.rental_id = i.rental_id
WHERE i.amount > 400

UNION ALL

--SELECT statement #2
SELECT c.first_name, c.last_name, c.phone_number, u.unit_number, f.city
FROM su_customer as c
JOIN su_rental as r ON c.customer_id = r.customer_id
JOIN su_unit as u ON r.unit_id = u.unit_id
JOIN su_facility as f ON u.facility_id = f.facility_id
WHERE r.discount_amount > 0

This return is a bit messy, so we can sort it by putting it in a subquery:

In [ ]:
SELECT *
FROM (
    --SELECT statement #1
    SELECT c.first_name, c.last_name, c.phone_number, u.unit_number, f.city
    FROM su_customer as c
    JOIN su_rental as r ON c.customer_id = r.customer_id
    JOIN su_unit as u ON r.unit_id = u.unit_id
    JOIN su_facility as f ON u.facility_id = f.facility_id
    JOIN su_invoice as i ON r.rental_id = i.rental_id
    WHERE i.amount > 400

    UNION ALL

    --SELECT statement #2
    SELECT c.first_name, c.last_name, c.phone_number, u.unit_number, f.city
    FROM su_customer as c
    JOIN su_rental as r ON c.customer_id = r.customer_id
    JOIN su_unit as u ON r.unit_id = u.unit_id
    JOIN su_facility as f ON u.facility_id = f.facility_id
    WHERE r.discount_amount > 0
) as sub
ORDER BY sub.last_name, sub.first_name, sub.unit_number

Now it is much easier to see duplicates and organize the data. While this is unnecessary, it is a good reminder that pretty much any SELECT statement can be used as a subqery.

  

Next we will use INTERSECT to see which records appear in both queries:

In [ ]:
--SELECT statement #1
SELECT c.first_name, c.last_name, c.phone_number, u.unit_number, f.city
FROM su_customer as c
JOIN su_rental as r ON c.customer_id = r.customer_id
JOIN su_unit as u ON r.unit_id = u.unit_id
JOIN su_facility as f ON u.facility_id = f.facility_id
JOIN su_invoice as i ON r.rental_id = i.rental_id
WHERE i.amount > 400

INTERSECT

--SELECT statement #2
SELECT c.first_name, c.last_name, c.phone_number, u.unit_number, f.city
FROM su_customer as c
JOIN su_rental as r ON c.customer_id = r.customer_id
JOIN su_unit as u ON r.unit_id = u.unit_id
JOIN su_facility as f ON u.facility_id = f.facility_id
WHERE r.discount_amount > 0

Finally, we can use EXCEPT to see which query result is in the first query, and not in the second query.

In [ ]:
--SELECT statement #1
SELECT c.first_name, c.last_name, c.phone_number, u.unit_number, f.city
FROM su_customer as c
JOIN su_rental as r ON c.customer_id = r.customer_id
JOIN su_unit as u ON r.unit_id = u.unit_id
JOIN su_facility as f ON u.facility_id = f.facility_id
JOIN su_invoice as i ON r.rental_id = i.rental_id
WHERE i.amount > 400

EXCEPT

--SELECT statement #2
SELECT c.first_name, c.last_name, c.phone_number, u.unit_number, f.city
FROM su_customer as c
JOIN su_rental as r ON c.customer_id = r.customer_id
JOIN su_unit as u ON r.unit_id = u.unit_id
JOIN su_facility as f ON u.facility_id = f.facility_id
WHERE r.discount_amount > 0

If we were to change the order of the queries, then we could see which values are in the second query that are not in the first. Set operators are another tool that we can use to make our sql script more powerful.

## **Window Functions**

Finally, window functions allow us to aggregate data without grouping rows together. This way we can keep all the individual data about a record, but still display aggregate information. To use window functions, we will need the following keywords:

- OVER- Defines the set of rows for the window function to operate on.
- PARTITION BY- Optional, divides the result set into partitions to which the window function is applied.

Below is a simple window function that shows a running total of the total\_amount across all rows of the ho\_reservation table, but first we will add a few more records.

In [ ]:
INSERT INTO ho_reservation VALUES
    (3, 3, '2024-07-03', '2024-07-07', 480.00),
    (4, 4, '2024-07-04', '2024-07-08', 480.00),
    (5, 5, '2024-07-05', '2024-07-10', 1000.00);

In [ ]:
SELECT 
    reservation_id,
    guest_id,
    room_id,
    check_in_date,
    total_amount,
    --Below is the window function
    --We include the ORDER BY clause below to ensure that the cumulative_total is updated at each individual reservation_id. It isn't necessarily because we want to order the data.
    SUM(total_amount) OVER (ORDER BY reservation_id) AS cumulative_total
FROM 
    ho_reservation;

We can see in the above cell that the window function we implemented is keeping a running total of the total\_amount for each row. The basic structure of a window function will remain consistent, it is always:

AGGREGATE(column) OVER (ORDER BY/PARTITION BY statement) AS new\_column\_name.

Next we will partition our data. This is like implementing a GROUP BY clause into our window function. We will use the Storage Unit system for this query.

In [ ]:
SELECT u.unit_number, t.length, t.width, t.height, i.amount,
AVG(i.amount) OVER (PARTITION BY t.width ORDER BY t.width) as average_for_width --The average_for_width field will help us compare the amount that we charged per a rental against the average charge for all rentals of that same width. This can help us see if we are under or over charging.
FROM su_type t
JOIN su_unit u ON t.type_id = u.unit_id
JOIN su_rental r ON u.unit_id = r.unit_id
JOIN su_invoice i ON r.rental_id = i.rental_id

The above query partitions the data by width so that the AVE(i.amount) is grouped by width. This means that all records with the same width, will have the same value for our newly calculated column. Window functions can seem tricky, but since they are just part of SELECT statements you can adjust your syntax as many times as needed without having to worry about making unintended changes to your database.

## **Exercises / Scenario**

For this scenario you have been tasked with standing up a small database system to help a college manage it's enrollment. The college has provided an ERD that you should use to design your tables, you can find it under _College ERD_ in github. Your tasks are as follows:

- Use the provided ERD to guide table creation
- Insert data into each of the tables
- Add indexes where appropriate
- Create queries with set operators
- Create queries with window functions

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">1. In the below cell, write a query that contains the CREATE statements necessary for instantiating the required tables:</span>

In [6]:
CREATE TABLE co_major (
    major_id INT IDENTITY(1,1) PRIMARY KEY,
    major_name VARCHAR(100) NOT NULL
);

CREATE TABLE co_instructor (
    instructor_id INT IDENTITY(1,1) PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL
);

CREATE TABLE co_student (
    student_id INT IDENTITY(1,1) PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    phone_number VARCHAR(15) NOT NULL,
    major_id INT,
    FOREIGN KEY (major_id) REFERENCES co_major(major_id)
);

CREATE TABLE co_course (
    course_id INT IDENTITY(1,1) PRIMARY KEY,
    subject VARCHAR(10) NOT NULL,
    course_number VARCHAR(10) NOT NULL,
    course_title VARCHAR(100) NOT NULL,
    credits INT NOT NULL,
    instructor_id INT,
    FOREIGN KEY (instructor_id) REFERENCES co_instructor(instructor_id)
);

CREATE TABLE co_registration (
    student_id INT,
    course_id INT,
    registration_date DATE NOT NULL,
    grade CHAR(1),
    PRIMARY KEY (student_id, course_id),
    FOREIGN KEY (student_id) REFERENCES co_student(student_id),
    FOREIGN KEY (course_id) REFERENCES co_course(course_id)
);


Commands completed successfully.

Total execution time: 00:00:00.142

2\. In the below cell, write a query to insert data into each of the tables. The data values that need inserted have been provided. You only need to construct the INSERT statements and transaction language needed to complete the operation. (Cell is condensed, you will need to expand.)

In [7]:
--Your code here
BEGIN TRANSACTION;

--Insert data into co_major
INSERT INTO co_major (major_name)
VALUES
    ('Computer Science'),
    ('Mathematics'),
    ('Physics'),
    ('Chemistry'),
    ('Biology'),
    ('English Literature'),
    ('History'),
    ('Economics');

--Insert data into co_student
INSERT INTO co_student (first_name, last_name, phone_number, major_id)
VALUES
    ('John', 'Doe', '1234567890', 1),
    ('Jane', 'Smith', '1234567891', 2),
    ('Alice', 'Johnson', '1234567892', 3),
    ('Bob', 'Williams', '1234567893', 4),
    ('Charlie', 'Brown', '1234567894', 5),
    ('David', 'Jones', '1234567895', 6),
    ('Eve', 'Garcia', '1234567896', 7),
    ('Frank', 'Miller', '1234567897', 8),
    ('Grace', 'Davis', '1234567898', 1),
    ('Hank', 'Martinez', '1234567899', 2),
    ('Ivy', 'Hernandez', '1234567890', 3),
    ('Jack', 'Lopez', '1234567891', 4),
    ('Karen', 'Gonzalez', '1234567892', 5),
    ('Leo', 'Wilson', '1234567893', 6),
    ('Mia', 'Anderson', '1234567894', 7),
    ('Nina', 'Thomas', '1234567895', 8),
    ('Oscar', 'Taylor', '1234567896', 1),
    ('Paul', 'Moore', '1234567897', 2),
    ('Quinn', 'Jackson', '1234567898', 3),
    ('Rick', 'Martin', '1234567899', 4),
    ('Sara', 'Lee', '1234567890', 5),
    ('Tom', 'Perez', '1234567891', 6),
    ('Uma', 'White', '1234567892', 7),
    ('Vince', 'Harris', '1234567893', 8),
    ('Wendy', 'Clark', '1234567894', 1),
    ('Xander', 'Lewis', '1234567895', 2),
    ('Yara', 'Robinson', '1234567896', 3),
    ('Zane', 'Walker', '1234567897', 4),
    ('Amy', 'Young', '1234567898', 5),
    ('Brian', 'King', '1234567899', 6),
    ('Chloe', 'Scott', '1234567890', 7),
    ('Dan', 'Green', '1234567891', 8),
    ('Ella', 'Adams', '1234567892', 1),
    ('Finn', 'Baker', '1234567893', 2),
    ('Gina', 'Carter', '1234567894', 3),
    ('Hugo', 'Evans', '1234567895', 4),
    ('Iris', 'Collins', '1234567896', 5),
    ('Jake', 'Wright', '1234567897', 6),
    ('Kara', 'Mitchell', '1234567898', 7),
    ('Liam', 'Perez', '1234567899', 8);

--Insert data into co_instructor
INSERT INTO co_instructor (first_name, last_name, email)
VALUES
    ('Ava', 'Allen', 'ava.allen@example.com'),
    ('Ben', 'Bennett', 'ben.bennett@example.com'),
    ('Clara', 'Clark', 'clara.clark@example.com'),
    ('Dylan', 'Davis', 'dylan.davis@example.com'),
    ('Ella', 'Edwards', 'ella.edwards@example.com'),
    ('Felix', 'Foster', 'felix.foster@example.com'),
    ('Gina', 'Gray', 'gina.gray@example.com'),
    ('Henry', 'Harris', 'henry.harris@example.com'),
    ('Isla', 'Irwin', 'isla.irwin@example.com'),
    ('Jack', 'Jones', 'jack.jones@example.com'),
    ('Luna', 'Lewis', 'luna.lewis@example.com'),
    ('Mason', 'Miller', 'mason.miller@example.com'),
    ('Nina', 'Nelson', 'nina.nelson@example.com'),
    ('Oscar', 'Owens', 'oscar.owens@example.com');

--Insert data into co_course
INSERT INTO co_course (subject, course_number, course_title, credits, instructor_id)
VALUES
    ('MATH', '101', 'Calculus I', 500, 1),
    ('MATH', '102', 'Calculus II', 600, 2),
    ('MATH', '201', 'Linear Algebra', 575, 3),
    ('ENGL', '101', 'English Composition', 400, 4),
    ('ENGL', '201', 'American Literature', 500, 5),
    ('SCI', '101', 'General Biology', 600, 6),
    ('SCI', '102', 'General Chemistry', 670, 7),
    ('SCI', '201', 'Physics I', 900, 8),
    ('CIS', '101', 'Intro to Programming', 300, 9),
    ('CIS', '102', 'Data Structures', 700, 10),
    ('CIS', '201', 'Databases', 450, 11),
    ('HIST', '101', 'World History', 450, 12),
    ('HIST', '201', 'American History', 590, 13),
    ('MATH', '103', 'Statistics', 375, 14),
    ('ENGL', '202', 'British Literature', 460, 1),
    ('SCI', '202', 'Physics II', 720, 2),
    ('CIS', '202', 'Operating Systems', 690, 3),
    ('HIST', '202', 'European History', 455, 4),
    ('MATH', '202', 'Differential Equations', 525, 5),
    ('ENGL', '203', 'Creative Writing', 415, 6),
    ('SCI', '203', 'Organic Chemistry', 600, 7),
    ('CIS', '203', 'Network Security', 680, 8),
    ('HIST', '203', 'Modern History', 375, 9),
    ('MATH', '301', 'Abstract Algebra', 500, 10),
    ('CIS', '301', 'Machine Learning', 700, 11);

--Insert data into co_registration
INSERT INTO co_registration (student_id, course_id, registration_date, grade)
VALUES
    (1, 1, '2024-01-10', 'A'),
    (1, 2, '2024-01-10', 'B'),
    (2, 1, '2024-01-10', 'A'),
    (2, 3, '2024-01-10', 'B'),
    (3, 2, '2024-01-10', 'A'),
    (3, 4, '2024-01-10', 'C'),
    (4, 3, '2024-01-10', 'B'),
    (4, 5, '2024-01-10', 'A'),
    (5, 4, '2024-01-10', 'A'),
    (5, 6, '2024-01-10', 'C'),
    (6, 5, '2024-01-10', 'B'),
    (6, 7, '2024-01-10', 'A'),
    (7, 6, '2024-01-10', 'A'),
    (7, 8, '2024-01-10', 'B'),
    (8, 7, '2024-01-10', 'C'),
    (8, 9, '2024-01-10', 'B'),
    (9, 8, '2024-01-10', 'A'),
    (9, 10, '2024-01-10', 'A'),
    (10, 9, '2024-01-10', 'B'),
    (10, 11, '2024-01-10', 'C'),
    (11, 10, '2024-01-10', 'A'),
    (11, 12, '2024-01-10', 'B'),
    (12, 11, '2024-01-10', 'C'),
    (12, 13, '2024-01-10', 'A'),
    (13, 12, '2024-01-10', 'B'),
    (13, 14, '2024-01-10', 'A'),
    (14, 13, '2024-01-10', 'A'),
    (14, 15, '2024-01-10', 'B'),
    (15, 14, '2024-01-10', 'C'),
    (15, 16, '2024-01-10', 'A'),
    (16, 15, '2024-01-10', 'B'),
    (16, 17, '2024-01-10', 'A'),
    (17, 16, '2024-01-10', 'A'),
    (17, 18, '2024-01-10', 'B'),
    (18, 17, '2024-01-10', 'C'),
    (18, 19, '2024-01-10', 'B'),
    (19, 18, '2024-01-10', 'A'),
    (19, 20, '2024-01-10', 'A'),
    (20, 19, '2024-01-10', 'B');

COMMIT;



(8 rows affected)

(40 rows affected)

(14 rows affected)

(25 rows affected)

(39 rows affected)

Total execution time: 00:00:01.975

3\. In the below cell, write a script to add the following unclustered indexes:

- co\_student.stu\_last\_name
- co\_course.subject
- co\_instructor.ins\_last\_name

In [8]:
--Your code here
CREATE INDEX idx_student_major ON co_student (major_id);
CREATE INDEX idx_course_instructor ON co_course (instructor_id);
CREATE INDEX idx_registration_date ON co_registration (registration_date);



Commands completed successfully.

Total execution time: 00:00:00.133

4A. In the below cell, write a query to select all of the students taking MATH (subject) courses. Include their full name, phone number, and major:

In [9]:
--Your code here
SELECT 
    s.first_name, 
    s.last_name, 
    s.phone_number, 
    m.major_name
FROM co_student s
JOIN co_registration r ON s.student_id = r.student_id
JOIN co_course c ON r.course_id = c.course_id
JOIN co_major m ON s.major_id = m.major_id
WHERE c.subject = 'MATH';



(10 rows affected)

Total execution time: 00:00:00.134

first_name,last_name,phone_number,major_name
John,Doe,1234567890,Computer Science
John,Doe,1234567890,Computer Science
Jane,Smith,1234567891,Mathematics
Jane,Smith,1234567891,Mathematics
Alice,Johnson,1234567892,Physics
Bob,Williams,1234567893,Chemistry
Karen,Gonzalez,1234567892,Biology
Mia,Anderson,1234567894,History
Paul,Moore,1234567897,Mathematics
Rick,Martin,1234567899,Chemistry


4B. In the below cell, write a query to select all of the students taking ENGL (subject) courses. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Include their full name, phone number, and major</span><span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">:</span>

In [10]:
--Your code here
SELECT 
    s.first_name, 
    s.last_name, 
    s.phone_number, 
    m.major_name
FROM co_student s
JOIN co_registration r ON s.student_id = r.student_id
JOIN co_course c ON r.course_id = c.course_id
JOIN co_major m ON s.major_id = m.major_id
WHERE c.subject = 'ENGL';

(7 rows affected)

Total execution time: 00:00:00.120

first_name,last_name,phone_number,major_name
Alice,Johnson,1234567892,Physics
Bob,Williams,1234567893,Chemistry
Charlie,Brown,1234567894,Biology
David,Jones,1234567895,English Literature
Leo,Wilson,1234567893,English Literature
Nina,Thomas,1234567895,Economics
Quinn,Jackson,1234567898,Physics


4C. In the below cell, use a set operator to select all student who are taking a math class, that are not taking an english class. Order the return alphabetically by last name then first name:

In [11]:
--Your code here
SELECT 
    s.first_name, 
    s.last_name
FROM co_student s
JOIN co_registration r ON s.student_id = r.student_id
JOIN co_course c ON r.course_id = c.course_id
WHERE c.subject = 'MATH'
AND s.student_id NOT IN (
    SELECT s2.student_id
    FROM co_student s2
    JOIN co_registration r2 ON s2.student_id = r2.student_id
    JOIN co_course c2 ON r2.course_id = c2.course_id
    WHERE c2.subject = 'ENGL'
)
ORDER BY s.last_name, s.first_name;



(8 rows affected)

Total execution time: 00:00:00.131

first_name,last_name
Mia,Anderson
John,Doe
John,Doe
Karen,Gonzalez
Rick,Martin
Paul,Moore
Jane,Smith
Jane,Smith


5\. In the below cell, write a query that shows the course subject, number, name, and tuition, as well as an average for tuition of courses with the same subject. Remember, you partition by the column that you want to group by.

First 5 rows of correct solution:
|subject|course_num|course_name|tuition|avg_tuition_per_subject|
|---|---|---|---|---|
|CIS|101|Intro to Programming|300|586|
|CIS|102|Data Structures|700|586|
|CIS|201|Databases|450|586|
|CIS|202|Operating Systems|690|586|
|CIS|203|Network Security|680|586|


In [12]:
--Your code here
SELECT 
    c.subject, 
    c.course_number, 
    c.course_title, 
    c.credits AS tuition, 
    AVG(c.credits) OVER (PARTITION BY c.subject) AS avg_tuition
FROM co_course c;




(25 rows affected)

Total execution time: 00:00:00.099

subject,course_number,course_title,tuition,avg_tuition
CIS,101,Intro to Programming,300,586
CIS,102,Data Structures,700,586
CIS,201,Databases,450,586
CIS,202,Operating Systems,690,586
CIS,203,Network Security,680,586
CIS,301,Machine Learning,700,586
ENGL,203,Creative Writing,415,443
ENGL,202,British Literature,460,443
ENGL,101,English Composition,400,443
ENGL,201,American Literature,500,443


6\. In the below cell, write a query to drop all of the college tables if they exist:

In [13]:
--Your code here
DROP TABLE IF EXISTS co_registration;
DROP TABLE IF EXISTS co_course;
DROP TABLE IF EXISTS co_student;
DROP TABLE IF EXISTS co_instructor;
DROP TABLE IF EXISTS co_major;



Commands completed successfully.

Total execution time: 00:00:00.131